In [1]:
import pandas as pd

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
pd.options.display.float_format = '{:.2f}'.format

# 사용자 설정

In [4]:
addInspAmnt = 1000    # 증원인원
valWgtFile = '변수 및 가중치 입력.xlsx'
#전제: 원본엑셀파일에 숫자는 데이터 형태를 다 숫자로 변경한 한후 프로그램을 실행하여야함
orgFile = "(250618)근로감독관 충원 RAW DATA(v0.3).xlsx"  

# 사용할 변수 및 가중치 가져오기

In [7]:
df_tmp = pd.read_excel(valWgtFile)[12:]

#### 가중치 세트별로 구분하기

In [8]:
wgtLst = df_tmp.columns[2:]  #가중치 세트 명칭만
df_tmpLst = []     # 변수-가중치1, 변수-가중치2... 데이터프레임을 저장하는 리스트
for w in wgtLst:
    df = df_tmp[df_tmp[w] > 0][['변수', w]]
    df.rename(columns = {w: '가중치'}, inplace=True)
    df_tmpLst.append(df)   # 가중치가 있는 변수만 추리기 

In [9]:

useValsLst = []
for df in df_tmpLst:
    useValsLst.append(dict(zip(df['변수'], df['가중치'])))

In [16]:
useValsLst[1]

{'신고사건접수건수': 0.2,
 '감시단속적근로자적용제외승인': 0.2,
 '근로감독건수': 0.4,
 '정기감독실시업체': 0.1,
 '안전보건진단·개선계획수립명령건수(개선계획)': 0.1}

# 감독관 1000명 증원

### 함수 정의

##### 프로세스 설명 : 1.감독관 1인당 담당량을 계산  2. 1인당 담당량이 가장 만은 지청에 감독관 1명을 증원  3.  1,2 반복


In [11]:
# df_증원 = df_org.copy()
# df_증원['반복회수'] = 0
# df_증원['최종감독관수'] = df_증원['감독관']

# df_증원 = df_증원[['반복회수'] + [col for col in df_증원.columns if col != '반복회수']]

# plus_arr = []
# df_증원log = pd.DataFrame(columns = df_증원.columns)
# df_증원기관log = pd.DataFrame(columns = ['반복회차', '기관명', '현감독관', '작업후_감독관'])

#### 1인당 업무량을 계산하는 함수 정의(정규화, 가중치 적용 작업 포함)

In [12]:

def calc_point(df_증원, valsLst):
    #1인당 각 변수별 업무량
    
    #print(df_증원)
    for val in valsLst.keys() :
        
        df_증원['1인_' + val] = df_증원[val]/df_증원['최종감독관수']

    # 0~100정규화, 가중치 적용
    scaler = MinMaxScaler()    
    for val in valsLst.keys() :
        df_증원['1인nw_' + val] = scaler.fit_transform(df_증원[['1인_' + val]])*100 * valsLst[val]    

    #1인당 전체 업무량 계산
    v = 0
    for val in valsLst.keys() :
        v += df_증원['1인nw_' + val]

    df_증원['1인_업무량'] = v
    
    return df_증원    

#### 1인당 업무량이 많은 조직에 감독관을 증원하는 작업 반복하고 그결과를 리턴하는 함수

In [13]:
def calc_allocInspector(df_org_, valsLst, iter):
    
    
    df_org = df_org_
        #반복회수 컬럼 생성
        
    df_org['반복회수'] = 0
    df_org['최종감독관수'] = df_org['감독관']  #감독관 증원 할 때마다 카운팅 목적

    df_org = df_org[['반복회수'] + [col for col in df_org.columns if col != '반복회수']]   # 반복회수 컬럼을 맨 앞으로

    plus_arr = []
    df_증원log = pd.DataFrame(columns = df_org.columns)
    df_증원기관log = pd.DataFrame(columns = ['반복회차', '기관명', '현감독관'])
    

    for i in range(0,iter):

        df_org = calc_point(df_org, valsLst)

        ### 1인당 업무량 높은 순 정렬
        df_org = df_org.sort_values(['1인_업무량'], ascending = False).copy().reset_index(drop=True)
        ### 상위 1번 지청에 감독관 1명 증원
        df_org.loc[0, '최종감독관수'] += 1
        
        df_org['반복회수'] = str(i+1) + '회 결과'
        

        #로그 저장
        #모든 데이터 log
        df_증원log = pd.concat([df_증원log, df_org],axis=0, sort=False)
        # 증원된 기관 log
        tmp = pd.DataFrame({'반복회차' : [i+1], 
                    '기관명':[df_org.loc[0, '기관명']], 
                    '현감독관':[df_org[df_org['기관명'] == df_org.loc[0, '기관명']].reset_index().loc[0, '감독관']],
                    '최종감독관수': [df_org.loc[0, '최종감독관수']]})
        df_증원기관log = pd.concat([df_증원기관log, tmp], axis=0, sort=False)
        
        plus_arr.append(df_org.loc[0, '기관명'])
        #print(df_증원기관)
    
    df_증원기관 = df_org.sort_values(['연번'])[['연번', '기관명', '감독관', '최종감독관수']].reset_index(drop=True)
    
    return df_증원log, df_증원기관log, df_증원기관


# 근로감독관 충원 RAW 데이터 가져오기

In [15]:
i = 1
for v in useValsLst: 

    # 적용할 변수의 값들을 가져온다.
    df_org = pd.read_csv("./(250618)근로감독관 충원 RAW DATA(v0.3).csv",  
                     usecols = ['연번','기관명','과수(전체)','전체정원','감독관'] +  list(v.keys()),
                     encoding = 'cp949')
    
    #전처리 NaN -> 0으로
    df_org = df_org.fillna(0)
    
#    print(df_org)
 #   print(v)
  #  print(Added)
    
           
    df_증원log, df_증원기관log, df_증원기관 = calc_allocInspector(df_org, v, addInspAmnt)
    
    fm = str(i) + "번째 가중치 결과물.xlsx"
    
    with pd.ExcelWriter(fm, engine='openpyxl') as writer:
        
        df_증원log.to_excel(writer, sheet_name = "반복회차별 결과 데이터", index=False)
        df_증원기관log.to_excel(writer, sheet_name = "반복회차별 증원된 기관", index=False)
        df_증원기관.to_excel(writer, sheet_name = "최종 기관별 증원 현황", index=False)
        pd.DataFrame([v]).to_excel(writer, sheet_name = "적용가중치", index=False)
        
    i += 1


C:\Users\Moel\AppData\Local\Temp\ipykernel_8964\2787517651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_증원['1인_' + val] = df_증원[val]/df_증원['최종감독관수']
C:\Users\Moel\AppData\Local\Temp\ipykernel_8964\2787517651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_증원['1인_' + val] = df_증원[val]/df_증원['최종감독관수']
C:\Users\Moel\AppData\Local\Temp\ipykernel_8964\2787517651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [ ]:
# df_증원log.to_excel("반복회차별 데이터.xlsx")
# df_증원기관log.to_excel("반복회차별 증원된 기관.xlsx")
# df_증원기관 = df_증원.sort_values(['연번'])[['기관명', '감독관', '최종감독관수']].reset_index(drop=True)
# df_증원기관.to_excel('최종 기관별 증원 수.xlsx')